Credit:
[MaskGCT GitHub](https://github.com/open-mmlab/Amphion/tree/main/models/tts/maskgct)

In [ ]:
base_path="/content"
# base_path="." #for  local devices

In [ ]:
#@title Install MaskGCT TTS and Restart Session [Cancel All Pop-Up] [Wait For 6 Minutes]
!rm -rf $base_path/Amphion
%cd $base_path
# !git clone https://github.com/open-mmlab/Amphion.git
!git clone https://github.com/NeuralFalconYT/Amphion
!pip install setuptools ruamel.yaml tqdm
# pip install tensorboard tensorboardX torch==2.0.1
!pip install transformers===4.41.1
!pip install -U encodec
!pip install black==24.1.1
!pip install oss2
!sudo apt-get install espeak-ng
!pip install phonemizer
!pip install g2p_en
!pip install accelerate==0.31.0
!pip install funasr zhconv zhon modelscope
# pip install git+https://github.com/lhotse-speech/lhotse
!pip install timm
!pip install jieba cn2an
!pip install unidecode
!pip install -U cos-python-sdk-v5
!pip install pypinyin
!pip install jiwer
!pip install omegaconf
!pip install pyworld
!pip install py3langid==0.2.2 LangSegment
# !pip install onnxruntime
!pip install onnxruntime-gpu==1.20.0
!pip install pyopenjtalk
!pip install pykakasi
!pip install -U openai-whisper
!pip install json5
!pip install pydub
!pip install gradio
# !pip install nltk
!pip install nltk==3.8.1
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

After auto-restarting the session, run from the next cell.

In [ ]:
base_path="/content"
# base_path="." #for  local devices
project_path=f"{base_path}/Amphion/"

In [ ]:
#@title Download Model
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import subprocess
import torch
import os
from pydub import AudioSegment
import uuid
import re
from huggingface_hub import hf_hub_download
semantic_code_ckpt,codec_encoder_ckpt,codec_decoder_ckpt,t2s_model_ckpt,s2a_1layer_ckpt,s2a_full_ckpt=None,None,None,None,None,None
def download_checkpoint():
  # download checkpoint
  global semantic_code_ckpt,codec_encoder_ckpt,codec_decoder_ckpt,t2s_model_ckpt,s2a_1layer_ckpt,s2a_full_ckpt
  # download semantic codec ckpt
  semantic_code_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="semantic_codec/model.safetensors"
  )
  # download acoustic codec ckpt
  codec_encoder_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="acoustic_codec/model.safetensors"
  )
  codec_decoder_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="acoustic_codec/model_1.safetensors"
  )
  # download t2s model ckpt
  t2s_model_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="t2s_model/model.safetensors"
  )
  # download s2a model ckpt
  s2a_1layer_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="s2a_model/s2a_model_1layer/model.safetensors"
  )
  s2a_full_ckpt = hf_hub_download(
      "amphion/MaskGCT", filename="s2a_model/s2a_model_full/model.safetensors"
  )
download_checkpoint()
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Utils

def get_max_gpu_memory():
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        total_memory = torch.cuda.get_device_properties(device).total_memory
        max_gpu_memory_gb = round(total_memory / (1024 ** 3), 2)  # Convert to GB and round to 2 decimal places
        # print(f"Maximum GPU memory available: {max_gpu_memory_gb} GB")
        return max_gpu_memory_gb
    else:
        print("CUDA is not available.")
        return None

def is_gpu_memory_over_limit(limit_gb=14.5):
    limit_gb=get_max_gpu_memory()-0.60
    if limit_gb==None:
      return False
    # Run nvidia-smi and capture the output
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                            stdout=subprocess.PIPE, text=True)

    # Split the result into lines (for each GPU if there are multiple)
    memory_used_mb_list = result.stdout.strip().splitlines()

    # Convert memory used from MB to GB and check each GPU's memory usage
    for i, memory_used_mb in enumerate(memory_used_mb_list):
        memory_used_gb = int(memory_used_mb) / 1024.0
        # print(f"GPU {i}: Current memory allocated: {memory_used_gb:.2f} GB")
        if memory_used_gb > limit_gb:
            # print(f"GPU {i} memory usage exceeds {limit_gb} GB.")
            return True

    # print("GPU memory usage is within safe limits.")
    return False




def is_audio_duration_greater_than(audio_path, max_duration):
    """Check if audio duration is greater than max_duration."""
    try:
        audio = AudioSegment.from_file(audio_path)
        duration = len(audio) / 1000  # Duration in seconds
        return duration > max_duration
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return False

def trim_and_convert_audio(input_audio_path, max_duration=30):
    """Trim audio to max_duration and convert to 16,000 Hz."""
    global base_path,reference_folder


    audio = AudioSegment.from_file(input_audio_path)
    trimmed_audio = audio[:max_duration * 1000] if len(audio) / 1000 > max_duration else audio
    # trimmed_audio = trimmed_audio.set_frame_rate(24000)

    # Generate output file name
    base_name = os.path.splitext(os.path.basename(input_audio_path))[0]
    output_file = f"{reference_folder}/{base_name}_final.wav"
    trimmed_audio.export(output_file, format="wav")

    return output_file

def process_audio(reference_audio, max_duration=15):
    global reference_folder
    """Process audio: trim if longer than max_duration and ensure 16,000 Hz."""
    if is_audio_duration_greater_than(reference_audio, max_duration):
        return trim_and_convert_audio(reference_audio, max_duration)

    # If audio is less than or equal to max_duration, convert to 16,000 Hz
    audio = AudioSegment.from_file(reference_audio)
    # audio = audio.set_frame_rate(24000)

    # Generate output file name
    base_name = os.path.splitext(os.path.basename(reference_audio))[0]
    output_file = f"{reference_folder}/{base_name}_final.wav"
    audio.export(output_file, format="wav")

    return output_file

def clean_file_name(file_path):
    # Get the base file name and extension
    file_name = os.path.basename(file_path)
    file_name, file_extension = os.path.splitext(file_name)

    # Replace non-alphanumeric characters with an underscore
    cleaned = re.sub(r'[^a-zA-Z\d]+', '_', file_name)

    # Remove any multiple underscores
    clean_file_name = re.sub(r'_+', '_', cleaned).strip('_')

    # Generate a random UUID for uniqueness
    random_uuid = uuid.uuid4().hex[:6]

    # Combine cleaned file name with the original extension
    clean_file_path = os.path.join(os.path.dirname(file_path), clean_file_name + f"_{random_uuid}" + file_extension)

    return clean_file_path


def tts_file_name(text):
    global save_audio_folder
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    file_name = f"{save_audio_folder}/{truncated_text}.wav"
    file_name=clean_file_name(file_name)
    return file_name

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def merge_audio(audio_list, save_path):
    # Initialize an empty audio segment
    merged_audio = AudioSegment.empty()

    # Loop through the list of audio files
    for audio_file in audio_list:
        # Load each audio file
        audio_segment = AudioSegment.from_wav(audio_file)
        # Append to the merged audio segment
        merged_audio += audio_segment

    # Export the merged audio to the specified save path
    merged_audio.export(save_path, format="wav")

def chunks_sentences(paragraph, join_limit=1):
    sentences = sent_tokenize(paragraph)
    # Initialize an empty list to store the new sentences
    new_sentences = []

    # Iterate through the list of sentences in steps of 'join_limit'
    for i in range(0, len(sentences), join_limit):
        # Join the sentences with a space between them
        new_sentence = ' '.join(sentences[i:i + join_limit])
        new_sentences.append(new_sentence)
    return new_sentences



from IPython.display import clear_output
clear_output()

In [ ]:
#@title load Model
import os
os.chdir(project_path)

import gc
import time
import torch
import safetensors
import soundfile as sf
from models.tts.maskgct.maskgct_utils import *
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import shutil
device = torch.device("cuda:0")

# Define global variables for models
semantic_model, semantic_codec, codec_encoder, codec_decoder, t2s_model = None, None, None, None, None
s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std = None, None, None, None
whisper_pipe,whisper_model=None,None
semantic_code_ckpt,codec_encoder_ckpt,codec_decoder_ckpt,t2s_model_ckpt,s2a_1layer_ckpt,s2a_full_ckpt

def clear_model_cache():
    """Clear global variables and GPU memory."""
    global semantic_model, semantic_codec, codec_encoder, codec_decoder
    global t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std
    global whisper_pipe, whisper_model

    # List of all global model variables
    model_vars = [
        "semantic_model", "semantic_codec", "codec_encoder", "codec_decoder",
        "t2s_model", "s2a_model_1layer", "s2a_model_full", "semantic_mean", "semantic_std",
        "whisper_pipe", "whisper_model"
    ]

    try:
        # Delete and set each variable to None
        for var in model_vars:
            # print(f"Variable: {var} before clearing: {globals()[var]}")  # Print before clearing
            if globals()[var] is not None:
                del globals()[var]
                globals()[var] = None
            # print(f"Variable: {var} after clearing: {globals()[var]}")  # Print after clearing

        # Clear Python garbage and GPU cache
        gc.collect()
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"Error while clearing cache: {e}")  # Print any error that occurs


# Load Whisper model

def load_model():
    """Load models into GPU memory, clearing old models first."""
    global semantic_model, semantic_codec, codec_encoder, codec_decoder
    global t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std
    global whisper_pipe,whisper_model
    global semantic_code_ckpt,codec_encoder_ckpt,codec_decoder_ckpt,t2s_model_ckpt,s2a_1layer_ckpt,s2a_full_ckpt
    # Clear any previously loaded models from memory
    clear_model_cache()
    time.sleep(5)  # Optional delay to ensure memory is fully cleared
    # Load configurations
    cfg_path = "./models/tts/maskgct/config/maskgct.json"
    cfg = load_config(cfg_path)

    # Load models
    semantic_model, semantic_mean, semantic_std = build_semantic_model(device)
    semantic_codec = build_semantic_codec(cfg.model.semantic_codec, device)
    codec_encoder, codec_decoder = build_acoustic_codec(cfg.model.acoustic_codec, device)
    t2s_model = build_t2s_model(cfg.model.t2s_model, device)
    s2a_model_1layer = build_s2a_model(cfg.model.s2a_model.s2a_1layer, device)
    s2a_model_full = build_s2a_model(cfg.model.s2a_model.s2a_full, device)

    # Load checkpoints
    safetensors.torch.load_model(semantic_codec, semantic_code_ckpt)
    safetensors.torch.load_model(codec_encoder, codec_encoder_ckpt)
    safetensors.torch.load_model(codec_decoder, codec_decoder_ckpt)
    safetensors.torch.load_model(t2s_model, t2s_model_ckpt)
    safetensors.torch.load_model(s2a_model_1layer, s2a_1layer_ckpt)
    safetensors.torch.load_model(s2a_model_full, s2a_full_ckpt)

    #load Whisper
    model_id = "openai/whisper-large-v3-turbo"
    whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True
    )
    whisper_model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    whisper_pipe = pipeline(
        "automatic-speech-recognition",
        model=whisper_model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch.float16,
        device=device,
    )

    return (semantic_model, semantic_codec, codec_encoder, codec_decoder,
            t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std,
            whisper_model,whisper_pipe)



def voice_clone(reference_path, text,duration=None):
    global semantic_model, semantic_codec, codec_encoder, codec_decoder
    global t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std
    global whisper_model, whisper_pipe
    prompt_wav_path = process_audio(reference_path, max_duration=15)
    if is_gpu_memory_over_limit():
        print("Loading models due to GPU memory limit")
        (
            semantic_model, semantic_codec, codec_encoder, codec_decoder,
            t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std,
            whisper_model, whisper_pipe
        ) = load_model()

    print("Processing reference audio:", prompt_wav_path)

    # Get the prompt text from the reference audio
    try:
        prompt_text = whisper_pipe(prompt_wav_path)['text'].strip()
    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

    print("Extracted prompt text:", prompt_text)

    # Set the target text
    target_text = text

    # Check if the prompt_text and target_text are not empty
    if not prompt_text or not target_text:
        print("Prompt text or target text is empty.")
        return None

    # Create the inference pipeline
    maskgct_inference_pipeline = MaskGCT_Inference_Pipeline(
        semantic_model,
        semantic_codec,
        codec_encoder,
        codec_decoder,
        t2s_model,
        s2a_model_1layer,
        s2a_model_full,
        semantic_mean,
        semantic_std,
        device,
    )

    try:
        recovered_audio = maskgct_inference_pipeline.maskgct_inference(
            prompt_wav_path, prompt_text, target_text, "en", "en", target_len=duration
        )
    except Exception as e:
        print(f"Error during inference: {e}")
        return None

    # Save the output audio
    save_path = tts_file_name(text)
    sf.write(save_path, recovered_audio, 24000)
    print(f"Audio saved to: {save_path}")
    return save_path



def voice_clone_long_text(reference_audio_path,text,input_duration):
  global base_path
  audio_list=[]
  temp_folder=f"{base_path}/clone_voice"
  if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)
  os.mkdir(temp_folder)
  sentences=chunks_sentences(text,join_limit=2)
  for index,sentence in enumerate(sentences):
    temp_path=voice_clone(reference_audio_path, sentence,duration=input_duration)
    temp_file=f"{temp_folder}/{index}.wav"
    shutil.move(temp_path,temp_file)
    audio_list.append(temp_file)
  cloned_voice_path=tts_file_name(text)
  merge_audio(audio_list, cloned_voice_path)
  return cloned_voice_path

def gradio_app(reference_audio_path,text,input_duration,large_text):
  if input_duration==-1 or input_duration==0:
    Duration=None
  else:
    Duration=input_duration
  if large_text:
    cloned_voice_path=voice_clone_long_text(reference_audio_path,text,Duration)
  else:
    cloned_voice_path=voice_clone(reference_audio_path, text,duration=Duration)
  return cloned_voice_path



# Call the function
# voice_clone("/content/reference.wav", "hi how are you guys",duration=None)


# Load models for the first time, clearing any old models
(
    semantic_model, semantic_codec, codec_encoder, codec_decoder,
    t2s_model, s2a_model_1layer, s2a_model_full, semantic_mean, semantic_std,
    whisper_model,whisper_pipe
) = load_model()


reference_folder = f"{base_path}/trim_audio"
save_audio_folder=f"{base_path}/clone_voice"
os.makedirs(reference_folder, exist_ok=True)
os.makedirs(save_audio_folder, exist_ok=True)



from IPython.display import clear_output
clear_output()

In [ ]:
#@title Using Gradio Interface
import gradio as gr


gradio_inputs=[gr.Audio(label="Reference Audio", type="filepath"),
               gr.Textbox(label="Text to Generate",lines=3),
               gr.Number(label="Target Duration (in seconds), if the target duration is less than 0, the system will estimate a duration.", value=-1),
               gr.Checkbox(label="Long Text ?",value=False)
              ]
gradio_outputs=[gr.Audio(label="Generated Audio")]
demo = gr.Interface(fn=gradio_app, inputs=gradio_inputs,outputs=gradio_outputs , title="MaskGCT TTS English Demo ")
demo.launch(allowed_paths=[f"{base_path}/clone_voice"],debug=False,share=True)


#Run From Colab Cell

In [ ]:
#@title Uplaod Reference Audio File
import os
from google.colab import files
from IPython.display import clear_output

def upload_audio():
    upload_folder = f"{base_path}/user_upload"

    # Ensure the upload folder exists
    os.makedirs(upload_folder, exist_ok=True)

    # Change to the upload directory
    os.chdir(upload_folder)

    # Upload the files
    uploaded = files.upload()

    # Switch back to the original install directory (ensure install_path is defined)
    os.chdir(project_path)

    audio_name_list = []

    for fn in uploaded.keys():
        file_path = f"{upload_folder}/{fn}"
        save_file_path = clean_file_name(file_path)

        # Rename the file with cleaned file name
        os.rename(file_path, save_file_path)
        audio_name_list.append(save_file_path)

    # Filter audio files based on valid audio extensions
    audio_extensions = ('.wav', '.mp3')
    valid_audio_files = [f for f in audio_name_list if f.lower().endswith(audio_extensions)]

    # Clear the output (for Google Colab)
    clear_output()

    if valid_audio_files:
        # Return the first valid audio file found
        return valid_audio_files[0]
    else:
        # Print message if no valid audio files were uploaded
        print("Please upload an audio file.")
        return None
upload_audio()

TTS_Duration: 0 mean None , the system will estimate a duration.


In [ ]:
#@title Generate TTS
Reference_Audio_Path= '/content/anime.mp3'  # @param {type: "string"}
Text_to_Generate= "What kind of story would you like? Fantasy, sci-fi, adventure, or maybe something heartwarming or mysterious? And do you want it to be short or longer? Let me know any details you want, and I'll dive right in!"  # @param {type: "string"}
TTS_Duration = 0  # @param {type: "number"}
Large_Text = 0  # @param {type: "boolean"}
cloned_voice_path=gradio_app(Reference_Audio_Path,Text_to_Generate,TTS_Duration,Large_Text)
from IPython.display import clear_output
clear_output()
print(f"TTS Save at {cloned_voice_path}")
from IPython.display import Audio
Audio(cloned_voice_path)

In [ ]:
#@title Download Cloned Voice
from google.colab import files
files.download(cloned_voice_path)